<a href="https://www.kaggle.com/code/jorgeluiscruzquispe/uniform-sampling?scriptVersionId=181160626" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## <center>Cantidad de muestreos de una zona</center>
---

In [1]:
!pip install -q leafmap
!pip install -q geemap

In [2]:
import geemap.foliumap as geemap
import leafmap.foliumap as leafmap
import geopandas as gpd
import shapely
import pandas as pd
import numpy as np
import ee

In [3]:
# funciones
def InfoPolygon(gdf_polygon):
    gdf_polygon.reset_index(drop = True, inplace = True)
    gdf_polygon = gpd.GeoDataFrame(gdf_polygon["geometry"])
    gdf_polygon[f"Numero ({len(gdf_polygon)})"] = 0
    gdf_polygon["Area (hec)"] = round(gdf_polygon.area/10000, 3)
    gdf_polygon["Perimetro (m)"] = round(gdf_polygon.length, 3)
    for j in range(len(gdf_polygon)):
        gdf_polygon.loc[j:j+1, f"Numero ({len(gdf_polygon)})"] = j + 1
    return gdf_polygon

def InfoPoint(gdf_point):
    gdf_point.reset_index()
    gdf_point = gpd.GeoDataFrame(gdf_point["geometry"])
    gdf_point[f"Numero ({len(gdf_point)})"] = 0
    for i in range(len(gdf_point)):
        gdf_point.loc[i:i + 1, f"Numero ({len(gdf_point)})"] = i + 1
    return gdf_point

def CalcPoints(area):
    df = pd.DataFrame({
        "Area (hec)": [0.1, 0.5, 1, 2, 3, 4, 5, 10, 15, 20, 25, 30, 40, 50, 100],
        "Puntos": [4, 6, 9, 15, 19, 21, 23, 30, 33, 36, 38, 40, 42, 44, 50],
    })

    df.set_index("Area (hec)", inplace = True)
    list1 = list(df.index)
    list2 = list(df.Puntos)
    if area in list1:
        return df.loc[area, "Puntos"]
    
    elif 0.1 < area < 100:
        i2 = 0
        for i in list1:
            if i > area:
                i2 = round(i)
                break
        position = list1.index(i2)
        i1 = list1[position - 1]
        p2 = list2[position]
        p1 = list2[position - 1]  
        return round(((p2 - p1) / (i2 - i1)) * (area - i1) + p1)
    
    elif 100 < area:
        return round(0.1*area + 40)
    
    else:
        return print("Ingresaste un valor no admisible")

In [4]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AdLIrYdjLw3N4K5vd_n549voDXiiGH9kqDdw0rCq0TvxezIkyG5tD52th40



Successfully saved authorization token.


In [5]:
area_estudio = shapely.geometry.Polygon([[-78.778818,-8.4033],[-78.778539,-8.404483],[-78.778391,-8.40456],
                                         [-78.776007,-8.404356],[-78.774403,-8.404255],[-78.773485,-8.40417],
                                         [-78.769784,-8.403883],[-78.76604,-8.403586],[-78.76618,-8.402367],
                                         [-78.767772,-8.402424],[-78.768408,-8.402498],[-78.77002,-8.402825],
                                         [-78.771045,-8.403018],[-78.771452,-8.403018],[-78.77212,-8.403002],
                                         [-78.773021,-8.402894],[-78.773327,-8.402879],[-78.775545,-8.403039],
                                         [-78.778818,-8.4033]])

area_gdf = gpd.GeoDataFrame(geometry = [area_estudio], crs = "EPSG:4326")

area_gdf.to_crs(32718, inplace = True)

area_ee = ee.Feature(geemap.gdf_to_ee(area_gdf).first())

point_gdf = gpd.GeoDataFrame(geometry = [area_gdf.centroid[0]], crs = "EPSG:32718")

point_gdf["Tipo"] = "Centroide del area de estudio"

area_gdf = InfoPolygon(area_gdf)

In [6]:
Map = leafmap.Map()

Map.add_basemap("HYBRID")

Map.add_gdf(point_gdf, style = {}, layer_name = "centroide")

Map.add_gdf(area_gdf, style = {"fillColor": "cyan", "color": "cyan"}, layer_name = "area estudio")

Map

In [7]:
points = CalcPoints(area_gdf["Area (hec)"][0])

print(f"Se requiere realizar {points} puntos de muestreo")

Se requiere realizar 35 puntos de muestreo


In [8]:
intervalos = 0.000655

fishnet = geemap.fishnet(area_ee,h_interval= intervalos, v_interval = intervalos, delta =1)

fishnet = geemap.ee_to_gdf(fishnet)

fishnet.to_crs(32718, inplace = True)

fishnet = gpd.GeoDataFrame(fishnet["geometry"])
    
geodata = gpd.GeoDataFrame(geometry = fishnet['geometry'].centroid)

clipped_data = gpd.overlay(geodata, area_gdf, how='intersection')

intersections = gpd.sjoin(fishnet, clipped_data, how='inner', op='intersects')

intersections = InfoPolygon(intersections)

clipped_data = InfoPoint(clipped_data)

print(f"Se tiene {len(clipped_data)} puntos de muestreo")

Se tiene 35 puntos de muestreo


In [9]:
Map = leafmap.Map()

Map.add_basemap("HYBRID")

Map.add_gdf(area_gdf, style = {"fillColor": "cyan", "color": "cyan"}, layer_name = "area de estudio")

Map.add_gdf(clipped_data, style = {"color": "red", "fillColor": 0, "shape": "circle", "icon": "circleMarker"}, 
            layer_name = "puntos 2")

Map.add_gdf(intersections, style = {"color": "red", "fillOpacity": 0}, layer_name = "areas divididas")

Map

In [10]:
df = clipped_data.bounds.iloc[:,0:2].rename(columns = {"minx": "Latitud", "miny": "Longitud"})
df.head()

,Latitud,Longitud
0,83742.999881,9.069005e+06
1,83815.258805,9.069006e+06
2,83887.517676,9.069006e+06
3,83959.776495,9.069007e+06
4,84032.035262,9.069008e+06


In [11]:
df.to_csv("coordenadas_muestreos.csv")